In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-24 08:31:42,093 INFO: Initializing external client


2025-01-24 08:31:42,098 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-24 08:31:44,418 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [4]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [5]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [6]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [7]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.05s) 


'2025-01-23 00:00:00 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [8]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [9]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  108185da60e9cf649981b894e0357488a7832882  -  2025-01-23 23:49:03+00:00


Processing commit:  cc76ee401c7aac5bfa1ece9947c824a52ad5b15e  -  2025-01-23 22:48:59+00:00


Processing commit:  29d12d60f9906bebd1ad64c68aaecf27ff2d070c  -  2025-01-23 21:49:07+00:00


Processing commit:  2e0b6edb5edb0ba5afffb613108425ad401756e7  -  2025-01-23 20:49:12+00:00


Processing commit:  5b9d1a9e5e270c76a0570ba2612a610b394c828e  -  2025-01-23 19:48:59+00:00


...

Processing commit:  2cb24046a69148d89d546f1db312ec2fa6f1d480  -  2025-01-23 03:49:05+00:00


Processing commit:  dc1fc22d41f2ad7e3fcf8704f4fef28fcc5525b0  -  2025-01-23 02:56:45+00:00


Processing commit:  2e4111639ef23e1b0425d1f21a75704588348bc1  -  2025-01-23 01:33:35+00:00


Processing commit:  fe38a332d2abda8b85071379873f282591bbfe9a  -  2025-01-23 00:49:42+00:00


breaking at:  2025-01-22 23:49:00+00:00
{'CLARENDON ROW': [[0, datetime.datetime(2025, 1, 23, 23, 49, 3, tzinfo=<git.objects.util.tzoffset object at 0x000002593017F130>)], [0, datetime.datetime(2025, 1, 23, 22, 48, 59, tzinfo=<git.objects.util.tzoffset object at 0x000002593021A4D0>)], [0, datetime.datetime(2025, 1, 23, 21, 49, 7, tzinfo=<git.objects.util.tzoffset object at 0x000002593021BC40>)], [0, datetime.datetime(2025, 1, 23, 20, 49, 12, tzinfo=<git.objects.util.tzoffset object at 0x000002593021BC10>)], [1, datetime.datetime(2025, 1, 23, 19, 48, 59, tzinfo=<git.objects.util.tzoffset object at 0x000002593023A440>)], [4, datetime.datetime(2025, 1, 23, 18, 49, 32, tzinfo=<git.objects.util.tzoffset object at 0x000002593023FA00>)], [15, datetime.datetime(2025, 1, 23, 17, 49, 13, tzinfo=<git.objects.util.tzoffset object at 0x000002593037D000>)], [14, datetime.datetime(2025, 1, 23, 16, 49, 54, tzinfo=<git.objects.util.tzoffset object at 0x00000259303865C0>)], [15, datetime.datetime(2025, 

In [10]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today['datetime'] = pd.to_datetime(df_bike_today['datetime'], utc=True).dt.floor('H') + datetime.timedelta(hours=1)
df_bike_today


,num_bikes_available,datetime,station
0,40.0,2025-01-24 00:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
1,40.0,2025-01-23 23:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
2,40.0,2025-01-23 22:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
3,40.0,2025-01-23 21:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
4,40.0,2025-01-23 20:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
5,40.0,2025-01-23 19:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
6,40.0,2025-01-23 18:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
7,40.0,2025-01-23 17:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
8,24.0,2025-01-23 16:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
9,26.0,2025-01-23 15:00:00+00:00,HEUSTON_BRIDGE_(NORTH)


## Fetch the weather data for the same time period

In [11]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}


Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-24 00:00:00+00:00,7.40,-0.305221,1.8,0.0,45.532528,30634.960938,6.0,dublin
1,2025-01-24 01:00:00+00:00,9.00,1.795629,1.4,0.0,44.056999,30634.960938,6.0,dublin
2,2025-01-24 02:00:00+00:00,10.85,3.973445,1.4,0.0,43.708508,30634.960938,6.0,dublin
3,2025-01-24 03:00:00+00:00,11.70,3.999386,1.4,0.0,49.209511,30634.960938,6.0,dublin
4,2025-01-24 04:00:00+00:00,10.75,1.553877,0.0,0.0,56.060680,30634.960938,6.0,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-02-02 19:00:00+00:00,6.40,2.590479,0.0,0.0,13.246826,32506.718750,0.0,dublin
236,2025-02-02 20:00:00+00:00,6.30,2.515190,0.0,0.0,12.738099,32506.718750,0.0,dublin
237,2025-02-02 21:00:00+00:00,6.25,2.492395,0.0,0.0,12.496718,32506.718750,0.0,dublin
238,2025-02-02 22:00:00+00:00,6.15,2.450578,0.0,0.0,11.988594,32506.718750,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [12]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 0.00% |                                      | Rows 0/24 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 58.33% |██████████████████▋             | Rows 14/24 | Elapsed Time: 00:01 | Remaining Time: 00:00

Uploading Dataframe: 100.00% |███████████████████████████████| Rows 24/24 | Elapsed Time: 00:01 | Remaining Time: 00:00

Launching job: bike_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [13]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 0.00% |                                     | Rows 0/240 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 240/240 | Elapsed Time: 00:00 | Remaining Time: 00:00

Launching job: weather_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions


2025-01-24 08:32:58,975 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED


2025-01-24 08:33:02,156 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


2025-01-24 08:34:53,844 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED


2025-01-24 08:34:53,999 INFO: Waiting for log aggregation to finish.


2025-01-24 08:35:02,565 INFO: Execution finished successfully.
